In [1]:
import loader
import argparse
import rnn_models
# from beam_search import *
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torch
from torchtext import data
from collections import defaultdict
import numpy as np
import pdb
import sacrebleu

from torchtext import data
from torchtext import datasets

import io
import os
import string

In [2]:
parser = argparse.ArgumentParser(description='Testing')
parser.add_argument("--max_sentence_length", help="maximum sentence length", type=int, default=50)
parser.add_argument("--min_freq", help="filter out tokens less than min frequency", type=int, default=3)
parser.add_argument("--max_vocab_size", help="at most n tokens in vocabulary", type=int, default=100000)

_StoreAction(option_strings=['--max_vocab_size'], dest='max_vocab_size', nargs=None, const=None, default=100000, type=<class 'int'>, choices=None, help='at most n tokens in vocabulary', metavar=None)

In [30]:
class Args():
    
    #########
    # Paths #
    #########
    
    data = '/scratch/vr1059/vi-en/'
    train_prefix = 'train_100'
    val_prefix = 'dev_100'
    test_prefix = 'test'
    src_ext = '.tok.vi'
    trg_ext = '.tok.en'

    max_sentence_length = 50
    min_freq = 3
    max_vocab_size = 100000
    
    ################
    # Model params #
    ################
    
    hidden_size = 256
    embedding_size = 256
    bidirectional = True
    num_encoder_layers = 2
    num_decoder_layers = 2
    attn_model = 'general'
    lr = 1e-3
    epochs = 2
    batch_size = 32
    print_every = 1
    clip = 1
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
train_data, val_data, test_data, src, trg = loader.load_data(args)

most common source vocabs: [('.', 105), (',', 73), ('của', 32), ('một', 28), ('bạn', 26), ('trong', 26), ('là', 25), ('những', 24), ('-', 24), ('và', 22)]
source vocab size: 155
most common english vocabs: [('.', 109), ('the', 90), (',', 86), ('to', 68), ('of', 59), ('and', 41), ('we', 41), ('you', 40), ('in', 38), ('that', 36)]
english vocab size: 139


In [20]:
print(len(train_data))
print(len(val_data))

100
100


In [21]:
def train_batch(phase, args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, batch, device):
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    ###########
    # Encoder #
    ###########
    
    seq_len, batch_size = batch.trg[0].shape
    hidden = encoder.random_init_hidden(device, batch_size)
    encoder_outputs, hidden = encoder(hidden, batch.src[0], batch.src[1])  
    
    ###########
    # Decoder #
    ###########
    
    # Teacher-forcing always ON
    
    # [2, 2, 2, ..., 2]. List of SOS tokens, batch-sized. 
    decoder_input = batch.trg[0][0,:] 
    eos_encountered_list = [False]*batch_size
    
    i = 0
    loss = 0
    number_of_loss_calculation = 0
    
    # decoder.hidden = encoder.hidden[:decoder.n_layers] 
    # Use last (forward) hidden state from encoder #TODO: verify
    hidden = hidden[:decoder.n_layers]
    
    while ((i+1 < seq_len) and (sum(eos_encountered_list) < batch_size)):
        
        logits, _, hidden = decoder(hidden, decoder_input, encoder_outputs)
        logits = logits.unsqueeze(0)
        class_probs = F.log_softmax(logits, dim = 2)
        decoder_input = batch.trg[0][i+1,:]
        
        # i+1 represents the current index in all sequences
        for j in range(batch_size):
            if not eos_encountered_list[j]:
                loss += loss_func(class_probs[0, j, :].view(1, -1), batch.trg[0][i+1, j].view(1))
                number_of_loss_calculation += 1
                
                if batch.trg[0][i+1, j] == EOS_IDX:
                    eos_encountered_list[j] = True
                    
        i += 1
        
     
    # calculate gradients on each parameter
    loss.backward()
    
    # clip if too large
    nn.utils.clip_grad_norm_(encoder.parameters(), args.clip)
    nn.utils.clip_grad_norm_(decoder.parameters(), args.clip)

    # take gradient step
    encoder_optimizer.step()
    decoder_optimizer.step()
        
    # report avg loss over minibatch
    return loss.item()/number_of_loss_calculation


#               #
# Loss function #
#               #

# loss += loss_func(output[0, j, :].view(1, -1), batch.trg[0][i+1, j].view(1))
                
# so the way NLLLoss is set up, the target is simply the index that you want to predict. 
# and the input can be a softmax over the entire output vocabulary space
# and nllloss calculate loss value between that index between predicted and 
# elementary vector e_target_idx (zeroes everywhere except 1 in target index position)
    

In [22]:
def train(args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, device, epoch_idx, 
                 train_data, val_data, trg):
    
    
    # Create batches with pre-sorted, similar-length sequences in each
    train_iter = data.BucketIterator(
        dataset=train_data, 
        batch_size=args.batch_size,
        repeat=False,
        sort_key=lambda x: len(x.src),
        sort_within_batch=True,
        device=device,
        train=True
    )

    # Set training flag
    encoder.train()
    decoder.train()

    train_losses = []
    for i, batch in enumerate(iter(train_iter)):
        avg_loss = train_batch('train', args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, batch, device)
        train_losses.append(avg_loss)
        if args.print_every and i % args.print_every == 0:
            print("train, epoch: {}, batch number: {}, batch loss: {}".format(
            epoch_idx, i, avg_loss))
            
    print("epoch: {}, average loss for epoch: {}, size of last batch {}".format(
    epoch_idx, np.mean(train_losses), batch.src[0].shape[1]))
        
    return np.mean(train_losses)

In [23]:
def calculate_bleu(predictions, labels):
    """
    Only pass a list of strings 
    """
    # n_gram = 4

    bleu = sacrebleu.raw_corpus_bleu(predictions, [labels], .01).score
    return bleu

In [9]:
calculate_bleu(['I am rich. '], ['I am rich.'])

100.00000000000004

In [24]:
def beam_search(decoder, decoder_input, encoder_outputs, hidden, max_length, k, trg):
    
    candidates = [(decoder_input, 0, hidden)]
    potential_candidates = []
    completed_translations = []

    # put a cap on sentence length
    for m in range(max_length):
        for c in candidates:
            # unpack the tuple
            c_sequence, c_score, c_hidden = c
            
            # EOS token
            if c_sequence[-1] == EOS_IDX:
                completed_translations.append((c_sequence, c_score))
                k = k - 1
            else:
                logits, _, hidden = decoder(c_hidden.contiguous()[:decoder.n_layers], c_sequence.contiguous()[-1].unsqueeze(0), encoder_outputs)
                next_word_probs = F.log_softmax(logits, dim = 1)
                # in the worst-case, one sequence will have the highest k probabilities
                # so to save computation, only grab the k highest_probability from each candidate sequence
                top_probs, top_idx = torch.topk(next_word_probs, k)
                top_probs.squeeze_()
                top_idx.squeeze_()
                for i in range(len(top_probs)):
                    word = top_idx[i].reshape(1, 1).to(device)
                    new_score = c_score + top_probs[i]
                    potential_candidates.append((torch.cat((c_sequence, word)).to(device), new_score, hidden))

        candidates = sorted(potential_candidates, key= lambda x: x[1], reverse=True)[0:k] 
        potential_candidates = []

    completed = completed_translations + candidates
    completed = sorted(completed, key= lambda x: x[1], reverse=True)[0] 
    return completed[0]

In [25]:
def ids_to_words(ids, trg):
    words = ""
    for x in ids:
        words += trg.vocab.itos[x.squeeze().item()] + ' '
    return words.strip()

In [12]:
# Chaitra said the best way to get hyperparameters for your model
# was to take like 100, 200, 500 examples from your dataset
# and see which hyperparameter combination overfits the best/fastest on that example. 
# And then use that as your hyperparameter combination to train the model. 

In [26]:
def val_batch(args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, batch, trg, device):
    
    encoder.eval()
    decoder.eval()
    
    ############
    #  encode  #
    ############
    
    _, batch_size = batch.trg[0].shape
    hidden = encoder.random_init_hidden(device, batch_size)
    encoder_outputs, hidden = encoder(hidden, batch.src[0], batch.src[1])
    
    #################
    #  beam search  #
    #################
    
    max_length = 30
    k = 2 
    
    translations = []
    trg_translations = []
    for i in range(batch_size):
        decoder_input = torch.tensor([[src.vocab.stoi['SOS']]], device=device)
        decoder_hidden = hidden[:, i, :].unsqueeze(1)
        encoder_outputs_i = encoder_outputs[:, i, :].unsqueeze(1)

        seq_of_ids = beam_search(decoder, decoder_input, encoder_outputs_i, decoder_hidden, max_length, k, trg)
        translations.append(ids_to_words(seq_of_ids, trg))
        trg_translations.append(ids_to_words(batch.trg[0][:batch.trg[1][i], i], trg))
        
    return translations, trg_translations 

In [27]:
def val(args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, device, epoch_idx, 
        val_data, trg):
    
    # Create minibatches over validation data
    val_iter = data.BucketIterator(
        dataset=val_data, 
        batch_size=args.batch_size,
        train=False,
        shuffle=False,
        # A key to use for sorting examples in order to batch together 
        # examples with similar lengths and minimize padding.
        sort=True,
        sort_key=lambda x: len(x.src),
        repeat=False,
        sort_within_batch=True,
        device=device
    )
    
    val_losses = []
    val_bleus = []
    val_references = []
    
    all_predicted = []
    all_trg = []
    for i, batch in enumerate(iter(val_iter)):
        predicted_trans, trg_trans = val_batch(args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, batch, trg, device)
#         print(calculate_bleu(predicted_trans, trg_trans))
        all_predicted += predicted_trans
        all_trg += trg_trans
    bleu = calculate_bleu(all_predicted, all_trg)
    print(bleu)
    return bleu
    
    
#                    #
# Batch & Dimensions #
#                    #
# `batch` represents a batch of examples. 
# `batch.src` consists of two tensors. 
# The first, `b.src[0]`, is the `src` examples from your batch; it's a tensor with the shape (max_seq_len, batch_size). 
# Your sequences have already been indexed and padded. 
# The second, `b.src[1]`, is the actual lengths of each sequence. It is of shape (batch_size, 1). 

# data.BucketIterator automatically batches sequences of similar lengths together. 
# it also automatically sorts in reverse order. 

# Say you have a bidirectional, 2-layer RNN encoder. A single batch has max length 19 and batch size 32. 
# The encoder_outputs will have shape: (19, 32, 512). 
# Basically, it only returns the topmost layer's hidden states at each step of the sequence. 
# And it concatenates both directional outputs (hidden states) for the topmost layer. 

In [31]:
src_padding_idx = src.vocab.stoi['<pad>']
trg_padding_idx = trg.vocab.stoi['<pad>']
EOS_IDX = trg.vocab.stoi['EOS']

encoder = rnn_models.Encoder(args, src_padding_idx, len(src.vocab)).to(device)
decoder = rnn_models.LuongAttnDecoderRNN(args, trg_padding_idx, len(trg.vocab)).to(device)

# initialize weights using gaussian with 0 mean and 0.01 std, just like the paper said
# TODO: Better initialization. Xavier?
for net in [encoder, decoder]:
    for name, param in net.named_parameters(): 
        #print(name, type(param), param)
        if 'bias' in name:
            nn.init.constant_(param, 0.0)
        elif 'weight' in name:
            nn.init.xavier_normal_(param)
            
encoder_optimizer = optim.Adam(encoder.parameters(), lr=args.lr)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=args.lr)

loss_func = nn.NLLLoss()

loss_history = []
bleu_history = []

for i in range(args.epochs):
    train_loss = train(args, encoder, decoder, encoder_optimizer, 
                                     decoder_optimizer, loss_func, device, i, 
                                    train_data, val_data, trg)
    
    loss_history.append(train_loss)
    bleu = val(args, encoder, decoder, encoder_optimizer, decoder_optimizer, loss_func, device, 0, val_data, trg)
    bleu_history.append(bleu)
    

train, epoch: 0, batch number: 0, batch loss: 4.935461492486339
train, epoch: 0, batch number: 1, batch loss: 4.80083347486413
train, epoch: 0, batch number: 2, batch loss: 4.66480396412037
train, epoch: 0, batch number: 3, batch loss: 3.791769509787088
epoch: 0, average loss for epoch: 4.548217110314482, size of last batch 4


TypeError: len() of a 0-d tensor